venecia

In [4]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Siréna (1947)", "Hamlet (1948)", "Manon (1949)", "Justice est faite (1950)", "Rashômon (1951)",
    "Jeux interdits (1952)", "Giulietta e Romeo (1954)", "Ordet (1955)", "Aparajito (1957)", "Muhomatsu No Issho (1958)",
    "La grande guerra (1959)", "Le passage du Rhin (1960)", "L'année dernière à Marienbad (1961)", "Cronaca familiare (1962)",
    "Le mani sulla città (1963)", "Deserto rosso (1964)", "Vaghe stelle dell'Orsa... (1965)", "La battaglia di Algeri (1966)",
    "Belle de jour (1967)", "Die Artisten in der Zirkuskuppel: ratlos (1968)", "Atlantic City, U.S.A. (1980)", "Gloria (1980)",
    "Die bleierne Zeit (1981)", "Der Stand der Dinge (1982)", "Prénom Carmen (1983)", "Rok spokojnego słońca (1984)",
    "Sans toit ni loi (1985)", "Le rayon vert (1986)", "Au revoir les enfants (1987)", "La leggenda del santo bevitore (1988)",
    "Beiqing chengshi (1989)", "Rosenkrantz And Guildenstern Are Dead (1990)", "Urga (1991)", "Qui Ju da guansi (1992)",
    "Short Cuts (1993)", "Trois couleurs: Bleu (1993)", "Aiqing Wansui (1994)", "Pred doždot (1994)", "Xich-lo (1995)",
    "Michael Collins (1996)", "Hana-bi (1997)", "Così ridevano (1998)", "Yi ge dou bu neng shao (1999)", "Dayereh (2000)",
    "Monsoon Wedding (2001)", "The Magdalene Sisters (2002)", "Vozvraščenie (2003)", "Vera Drake (2004)", "Brokeback Mountain (2005)",
    "Sanxia haoren (2006)", "Se, jie (2007)", "The Wrestler (2008)", "Lebanon (2009)", "Somewhere (2010)", "Faust (2011)",
    "Pietà (2012)", "Sacro GRA (2013)", "En duva satt på en gren och funderade på tillvaron (2014)", "Desde allá (2015)",
    "Ang babaeng humayo (2016)", "The Shape of Water (2017)", "Roma (2018)", "Joker (2019)", "Nomadland (2020)", "L'Événement (2021)",
    "All the Beauty and the Bloodshed (2022)", "Poor Things (2023)", "The Room Next Door (2024)"
]

peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\venezia.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'Siréna (1947)': Short, Drama, Horror
Generos para 'Hamlet (1948)': Drama
Generos para 'Manon (1949)': Romance, Adventure, Suspense, Drama, Crime film, Thriller, Mystery, Detective fiction, crime fiction, Police procedural
Generos para 'Justice est faite (1950)': Documentary, Short
Generos para 'Rashômon (1951)': Mystery
Generos para 'Jeux interdits (1952)': Adult
Generos para 'Giulietta e Romeo (1954)': Romance, Tragedy, Drama
Generos para 'Ordet (1955)': Drama
Generos para 'Aparajito (1957)': Drama
Generos para 'Muhomatsu No Issho (1958)': Drama, Romance
Generos para 'La grande guerra (1959)': War
Generos no encontrados a través de Google y OMDb para 'Le passage du Rhin (1960)', probamos con TMDb...
Generos no encontrados a través de TMDb para 'Le passage du Rhin (1960)', probamos con IMDb...
Pelicula 'Le passage du Rhin (1960)' no encontrada en IMDb!
Generos no encontrados para 'Le passage du Rhin (1960)'.
Generos para 'L'année dernière à Marienbad (1961)': Horror, Susp

goya

In [5]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "El viaje a ninguna parte (1986)", "El bosque animado (1987)", "Mujeres al borde de un ataque de 'nervios' (1988)", 
    "El sueño del mono loco (1989)", "¡Ay, Carmela! (1990)", "Amantes (1991)", "Belle Epoque (1992)", "Todos a la cárcel (1993)", 
    "Días contados (1994)", "Nadie hablará de nosotras cuando hayamos muerto (1995)", "Tesis (1996)", "La buena estrella (1997)", 
    "La niña de tus ojos (1998)", "Todo sobre mi madre (1999)", "El Bola (2000)", "Los otros (2001)", "Los lunes al sol (2002)", 
    "Te doy mis ojos (2003)", "Mar adentro (2004)", "La vida secreta de las palabras (2005)", "Volver (2006)", "La soledad (2007)", 
    "Camino (2008)", "Celda 211 (2009)", "Pa negre (2010)", "No habrá paz para los malvados (2011)", "Blancanieves (2012)", 
    "Vivir es fácil con los ojos cerrados (2013)", "La isla mínima (2014)", "Tarde para la ira (2016)", "The Bookshop (La librería) (2017)", 
    "Campeones (2018)", "Dolor y gloria (2019)", "Las niñas (2020)", "El buen patrón (2021)", "As bestas (2022)", 
    "La sociedad de la nieve (2023)", "El 47 (2024)", "La infiltrada (2024)"
]


peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\goya.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'El viaje a ninguna parte (1986)': Comedy, Drama
Generos no encontrados a través de Google y OMDb para 'El bosque animado (1987)', probamos con TMDb...
Generos no encontrados a través de TMDb para 'El bosque animado (1987)', probamos con IMDb...
Pelicula 'El bosque animado (1987)' no encontrada en IMDb!
Generos no encontrados para 'El bosque animado (1987)'.
Generos para 'Mujeres al borde de un ataque de 'nervios' (1988)': Romance, Comedy, Dark comedy, Drama, Comedy drama, Tragicomedy
Generos para 'El sueño del mono loco (1989)': Horror, Romance, Melodrama, Mystery, Suspense, Drama, Thriller, Erotic thriller
Generos para '¡Ay, Carmela! (1990)': Comedy, Drama, War
Generos para 'Amantes (1991)': Action, Romance, Drama, Melodrama, Adventure, Noir, Thriller, Mystery, crime fiction
Generos para 'Belle Epoque (1992)': Comedy, Drama, Romance
Generos para 'Todos a la cárcel (1993)': Comedy, Drama
Generos para 'Días contados (1994)': Drama
Generos para 'Nadie hablará de nosotras cu

cannes

In [6]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Anora (2024)", "Anatomy of a Fall (2023)", "Triangle of Sadness (2022)", "Titane (2021)", "Parasite (2019)", 
    "Shoplifters (2018)", "The Square (2017)", "I, Daniel Blake (2016)", "Dheepan (2015)", "Winter Sleep (2014)", 
    "Blue Is the Warmest Color (2013)", "Love (2012)", "The Tree of Life (2011)", "Uncle Boonmee Who Can Recall His Past Lives (2010)", 
    "The White Ribbon (2009)", "The Class (Entre les murs) (2008)", "4 Months, 3 Weeks & 2 Days (2007)", 
    "The Wind that Shakes the Barley (2006)", "The Child (2005)", "Fahrenheit 9/11 (Fahrenheit 911) (2004)", 
    "Elephant (2003)", "The Pianist (2002)", "The Son's Room (2001)", "Dancer in the Dark (2000)", "Rosetta (1999)", 
    "Eternity and a Day (1998)", "Taste of Cherry (1997)", "The Eel (1997)", "Secrets and Lies (1996)", "Underground (1995)", 
    "Pulp Fiction (1994)", "The Piano (1993)", "Farewell My Concubine (1993)", "The Best Intentions (1992)", 
    "Barton Fink (1991)", "Wild at Heart (1990)", "Sex, Lies and Videotape (1989)", "Pelle the Conqueror (1988)", 
    "Under the Sun of Satan (1987)", "The Mission (1986)", "When Father Was Away on Business (1985)", "Paris, Texas (1984)", 
    "The Ballad of Narayama (1983)", "The Road (1982)", "Missing (1982)", "Man of Iron (1981)", "All That Jazz (1980)", 
    "Kagemusha the Shadow Warrior (1980)", "The Tin Drum (1979)", "Apocalypse Now (1979)", "The Tree of Wooden Clogs (1978)", 
    "Padre padrone (1977)", "Taxi Driver (1976)", "Chronicle of the Burning Years (1975)", "The Conversation (1974)", 
    "The Hireling (1973)", "Scarecrow (1973)", "The Working Class Goes to Heaven (1972)", "The Mattei Affair (1972)", 
    "The Go-Between (1971)", "M.A.S.H. (1970)", "If.... (1969)", "Blow-Up (1967)", "A Man and a Woman (1966)", 
    "The Birds, the Bees and the Italians (1966)", "The Knack... and How to Get It (1965)", "The Umbrellas of Cherbourg (1964)", 
    "The Leopard (1963)", "The Given Word (Keeper of Promises) (1962)", "Viridiana (1961)", "Such a Long Absence (The Long Absence) (1961)", 
    "La Dolce Vita (The Sweet Life) (1960)", "Black Orpheus (1959)", "The Cranes are Flying (1958)", "Friendly Persuasion (1957)", 
    "The Silent World (1956)", "Marty (1955)", "Gate of Hell (1954)", "The Wages of Fear (1953)", 
    "The Tragedy of Othello: The Moor of Venice (1952)", "Two Cents Worth of Hope (1952)", "Miracle in Milan (1951)", 
    "Miss Julie (1951)", "The Third Man (1949)"
]


peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\cannes.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'Anora (2024)': Romance, Comedy, Drama
Generos para 'Anatomy of a Fall (2023)': Thriller, Drama, Legal drama, Narrative, Suspense, Crime film, Mystery, Psychological thriller, Detective fiction, crime fiction, Police procedural
Generos para 'Triangle of Sadness (2022)': Comedy, Satire, Dark comedy, Tragicomedy, Narrative, Drama
Generos para 'Titane (2021)': Drama, Horror, Sci-Fi
Generos para 'Parasite (2019)': Horror, Thriller, Comedy, Dark comedy, Crime, Suspense, Tragicomedy, Drama, crime fiction
Generos para 'Shoplifters (2018)': Comedy, Family, Children's film, Documentary, Crime, Drama, Crime film, Thriller, Mystery, crime fiction, Family Drama
Generos para 'The Square (2017)': Comedy, Satire, Dark comedy, Tragicomedy, Drama, Narrative
Generos para 'I, Daniel Blake (2016)': Drama
Generos para 'Dheepan (2015)': Action, Suspense, Drama, Crime film, Thriller, Mystery, crime fiction
Generos para 'Winter Sleep (2014)': Drama
Generos para 'Blue Is the Warmest Color (2013)':

san sebastian

In [7]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Tardes de Soledad (2024)", "O Corno (2023)", "Los Reyes del Mundo (2022)", "Crai Nou (2021)", 
    "Dasatskisi (2020)", "Pacificado (2019)", "Entre dos aguas (2018)", "The Disaster Artist (2017)", 
    "Wo Bu Shi Pan Jinlian (2016)", "Sparrows (2015)", "Magical Girl (2014)", "Pelo Malo (2013)", 
    "Dans la Maison (2012)", "Los Pasos Dobles (2011)", "Neds (2010)", "City of Life and Death (2009)", 
    "Pandoranin Kutusu (2008)", "A Thousand Years of Good Prayers (2007)", "Niwemang (2006)", 
    "Mon Fils à Moi (2006)", "Stesti (2005)", "Turtles Can Fly (2004)", "Schussangst (2003)", 
    "Los Lunes al Sol (2002)", "Taxi para Tres (2001)", "La Perdición de los Hombres (2000)", 
    "C'est quoi la vie? (1999)", "El Viento Se Llevó Lo Que (1998)", "Rien ne Va Plus (1997)", 
    "Bwana (1996)", "Trojan Eddie (1996)", "Margaret's Museum (1995)", "Días Contados (1994)", 
    "Principio y Fin (1993)", "Sara (1993)", "Un Lugar en el Mundo (1992)", "Alas de Mariposa (1991)", 
    "Las Cartas de Alou (1990)", "Homer and Eddie (1989)", "La Nación Clandestina (1989)", "On the Black Hill (1988)", 
    "Noce en Galilée (1987)", "La Mitad del Cielo (1986)", "Yesterday (1985)", "Rumble Fish (1984)", 
    "Coup de Foudre (1983)", "Demonios en el Jardín (1982)", "Storie di Ordinaria Follia (1981)", "Dyrygent (1980)", 
    "Ossenij Marafon (1979)", "Alambrista (1978)", "Mekhanicheskogo Pianino (1977)", "Tabor Ujodit v Niebo (1976)", 
    "Furtivos (1975)", "Badlands (1974)", "El Espíritu de la Colmena (1973)", "The Glass House (1972)", 
    "Le Genou de Claire (1971)", "Ondata di Calore (1970)", "The Rain People (1969)", "The Long Day's Dying (1968)", 
    "Two for the Road (1967)", "I Was Happy Here (1966)", "Mirage (1965)", "Zlatá Reneta (1965)", "America, America (1964)", 
    "Mafioso (1963)", "L'Isola di Arturo (1962)", "One-Eyed Jacks (1961)", "Romeo, Juliet a Tma (1960)"
]



peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\sansebastian.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos no encontrados a través de Google y OMDb para 'Tardes de Soledad (2024)', probamos con TMDb...
Generos no encontrados a través de TMDb para 'Tardes de Soledad (2024)', probamos con IMDb...
Pelicula 'Tardes de Soledad (2024)' no encontrada en IMDb!
Generos no encontrados para 'Tardes de Soledad (2024)'.
Generos no encontrados a través de Google y OMDb para 'O Corno (2023)', probamos con TMDb...
Generos no encontrados a través de TMDb para 'O Corno (2023)', probamos con IMDb...
Pelicula 'O Corno (2023)' no encontrada en IMDb!
Generos no encontrados para 'O Corno (2023)'.
Generos para 'Los Reyes del Mundo (2022)': Action, Romance, Comedy, Adventure, Drama
Generos para 'Crai Nou (2021)': Drama, Narrative
Generos para 'Dasatskisi (2020)': Drama, Narrative
Generos no encontrados a través de Google y OMDb para 'Pacificado (2019)', probamos con TMDb...
Generos no encontrados a través de TMDb para 'Pacificado (2019)', probamos con IMDb...
Pelicula 'Pacificado (2019)' no encontrada en IM

oscar

In [8]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Wings (1929)", "All Quiet on the Western Front (1930)", "Cimarron (1931)", "Grand Hotel (1932)", 
    "Cavalcade (1933)", "It Happened One Night (1934)", "Mutiny on the Bounty (1935)", "The Great Ziegler (1936)", 
    "The Life of Emile Zola (1937)", "You Can't Take It with You (1938)", "Gone with the Wind (1939)", 
    "Rebecca (1940)", "How Green Was My Valley (1941)", "Mrs. Miniver (1942)", "Casablanca (1943)", 
    "Going My Way (1944)", "The Lost Weekend (1945)", "The Best Years of Our Lives (1946)", 
    "Gentleman's Agreement (1947)", "Hamlet (1948)", "All the King's Men (1949)", "All About Eve (1950)", 
    "An American in Paris (1951)", "The Greatest Show on Earth (1952)", "From Here to Eternity (1953)", 
    "On the Waterfront (1954)", "Marty (1955)", "Around the World in 80 Days (1956)", "The Bridge on the River Kwai (1957)", 
    "Gigi (1958)", "Ben-Hur (1959)", "The Apartment (1960)", "West Side Story (1961)", "Lawrence of Arabia (1962)", 
    "Tom Jones (1963)", "My Fair Lady (1964)", "The Sound of Music (1965)", "A Man for All Seasons (1966)", 
    "The Graduate (1967)", "Oliver! (1968)", "Midnight Cowboy (1969)", "Patton (1970)", "The French Connection (1971)", 
    "The Godfather (1972)", "The Sting (1973)", "The Godfather Part II (1974)", "One Flew Over the Cuckoo's Nest (1975)", 
    "Rocky (1976)", "Annie Hall (1977)", "The Deer Hunter (1978)", "Kramer vs. Kramer (1979)", "Ordinary People (1980)", 
    "Chariots of Fire (1981)", "Gandhi (1982)", "Terms of Endearment (1983)", "Amadeus (1984)", "Out of Africa (1985)", 
    "Platoon (1986)", "The Last Emperor (1987)", "Rain Man (1988)", "Driving Miss Daisy (1989)", "Dances with Wolves (1990)", 
    "The Silence of the Lambs (1991)", "Unforgiven (1992)", "Schindler's List (1993)", "Forrest Gump (1994)", 
    "Braveheart (1995)", "The English Patient (1996)", "Titanic (1997)", "Shakespeare in Love (1998)", 
    "American Beauty (1999)", "Gladiator (2000)", "A Beautiful Mind (2001)", "Chicago (2002)", 
    "The Lord of the Rings: The Return of the King (2003)", "Million Dollar Baby (2004)", "Crash (2005)", 
    "The Departed (2006)", "No Country for Old Men (2007)", "Slumdog Millionaire (2008)", "The Hurt Locker (2009)", 
    "The King's Speech (2010)", "The Artist (2011)", "Argo (2012)", "12 Years a Slave (2013)", "Birdman (2014)", 
    "Spotlight (2015)", "La La Land (2016) (annullato e vinto da Moonlight dopo un errore)", "The Shape of Water (2017)", 
    "Green Book (2018)", "Parasite (2019)", "Nomadland (2020)", "CODA (2021)", "Everything Everywhere All at Once (2022)"
]



peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\oscar.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'Wings (1929)': Drama, Romance, War
Generos para 'All Quiet on the Western Front (1930)': War, Anti-war, Adventure, Drama, Silent
Generos para 'Cimarron (1931)': Drama, Western
Generos para 'Grand Hotel (1932)': Romance, Comedy, Melodrama, Drama, Black-and-white, crime fiction
Generos para 'Cavalcade (1933)': Drama, Romance, War
Generos para 'It Happened One Night (1934)': Screwball comedy, Action, Romance, Comedy, Romantic comedy, Melodrama, Road, Drama, Classic
Generos para 'Mutiny on the Bounty (1935)': Romance, History, Melodrama, Adventure, Drama, Black-and-white
Generos para 'The Great Ziegler (1936)': Romance, Comedy, Musical, Melodrama, Drama, Black-and-white, Historical drama, Classic, Musical Drama
Generos para 'The Life of Emile Zola (1937)': Drama, History, Black-and-white, Political drama
Generos para 'You Can't Take It with You (1938)': Romance, Comedy, Screwball comedy, Romantic comedy, Melodrama, Drama, Classic
Generos para 'Gone with the Wind (1939)': Hist

top50esp

In [9]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Avatar (2009)", "8 Apellidos Vascos (2014)", "Avatar: The Way of Water (2022)", "Inside Out 2 (2024)", 
    "Titanic (1997)", "Lo Imposible (2012)", "The Lion King (2019)", "Ocho Apellidos Catalanes (2015)", 
    "The Lord of the Rings: The Return of the King (2003)", "Barbie (2023)", "Star Wars: The Force Awakens (2015)", 
    "The Lord of the Rings: The Fellowship of the Ring (2001)", "Joker (2019)", "The Lord of the Rings: The Two Towers (2002)", 
    "Avengers: Endgame (2019)", "Spider-Man: No Way Home (2024)", "Spider-Man: No Way Home (2021)", 
    "Harry Potter and the Philosopher's Stone (2001)", "Pirates of the Caribbean: Dead Man's Chest (2006)", 
    "Shrek 2 (2004)", "Bohemian Rhapsody (2018)", "Super Mario Bros. The Movie (2023)", "Others, The (2001)", 
    "The Da Vinci Code (2006)", "The Sixth Sense (1999)", "A Monster Calls (2016)", "Aladdin (2019)", 
    "The Orphanage (2007)", "Toy Story 3 (2010)", "Up (2009)", "Star Wars: The Phantom Menace (1999)", 
    "Jurassic World: Fallen Kingdom (2018)", "Minions (2015)", "Deadpool & Wolverine (2024)", 
    "Harry Potter and the Chamber of Secrets (2002)", "Finding Nemo (2003)", "Mortadelo and Filemon, The (2003)", 
    "The Hobbit: An Unexpected Journey (2012)", "Pirates of the Caribbean: At World's End (2007)", 
    "Spider-Man (2024)", "Alice in Wonderland (2010)", "Spider-Man (2002)", 
    "The Twilight Saga: Breaking Dawn – Part 2 (2012)", "Shrek the Third (2007)", 
    "Pirates of the Caribbean: The Curse of the Black Pearl (2003)", "Torrente 2: Mission in Marbella (2001)", 
    "Toy Story 4 (2019)", "Beauty and the Beast (2017)", "Jurassic World (2015)", "Inside Out (2015)"
]



peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\topboxesp.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'Avatar (2009)': Action, Science fiction, Fantasy, Adventure, Drama, Classic
Generos para '8 Apellidos Vascos (2014)': Romance, Comedy, Melodrama, Drama
Generos para 'Avatar: The Way of Water (2022)': Animation, Action, Science fiction, Drama, Adventure, Thriller, Fantasy
Generos para 'Inside Out 2 (2024)': Animation, Children's film, Comedy, Adventure, Family film, Coming-of-age story, Drama
Generos para 'Titanic (1997)': Romance, Action, Disaster, Melodrama, Adventure, Historical drama, Drama, Thriller, Costume drama
Generos para 'Lo Imposible (2012)': History, Disaster, Documentary, Drama, Thriller, Adventure, Mystery, Historical Fiction
Generos para 'The Lion King (2019)': Animation, Children's film, Action, Comedy, Musical, Family film, Adventure, Drama, Musical Drama
Generos para 'Ocho Apellidos Catalanes (2015)': Romance, Comedy, Melodrama, Drama
Generos para 'The Lord of the Rings: The Return of the King (2003)': Action, Adventure, Fantasy, Science fiction, Drama
G

top50w

In [10]:
import requests
import json
import csv
from bs4 import BeautifulSoup

def obtener_generos_tmdb(titulo_pelicula):
    tmdb_api_key = "8de01cb9beb4cd3ac1547b4829959086"
    url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={titulo_pelicula}"
    response = requests.get(url)
    data = response.json()
    generos = []
    if data['results']:
        movie_id = data['results'][0]['id']
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()
        generos = [genero['name'] for genero in movie_data['genres']]
    return generos

def obtener_generos_imdb(titulo_pelicula):
    url = f"https://www.imdb.com/find?q={titulo_pelicula.replace(' ', '+')}&s=tt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movie_link_tag = soup.find('td', class_='result_text')
    if not movie_link_tag:
        print(f"Pelicula '{titulo_pelicula}' no encontrada en IMDb!")
        return []
    movie_link = movie_link_tag.find('a')['href']
    movie_url = f"https://www.imdb.com{movie_link}"
    movie_response = requests.get(movie_url)
    movie_soup = BeautifulSoup(movie_response.text, 'html.parser')
    generos = []
    genre_tags = movie_soup.find_all('span', class_='sc-16ede3d4-2')
    for tag in genre_tags:
        generos.append(tag.text)
    return generos

def obtener_generos_de_google(titulo_pelicula):
    api_key = "cb2a9dd654fa0ca4bf3a515fb09bb96b9995c581cf2de293664e893773e56553"
    params = {
        "q": f'"{titulo_pelicula} genre"',
        "api_key": api_key
    }
    url = "https://serpapi.com/search"
    response = requests.get(url, params=params)
    resultados = response.json()

    generos = []
    try:
        json_data = json.dumps(resultados, indent=2)
        start_index = json_data.find('"genres": [')
        end_index = json_data.find('],\n    "watch_now": [', start_index)
        
        if start_index != -1 and end_index != -1:
            generos_content = json_data[start_index + len('"genres": ['):end_index].strip()
            generos_data = json.loads(f"[{generos_content}]")
            generos = [genero["name"] for genero in generos_data]

    except Exception as e:
        print(f"Error durante el procesamiento de los datos para '{titulo_pelicula}' con Google: {e}")
    
    return generos

def obtener_generos_de_omdb(titulo_pelicula):
    omdb_api_key = "d2ebbf03"
    titulo_pelicula_sin_ano = titulo_pelicula.split("(")[0].strip()
    url = f"https://www.omdbapi.com/?t={titulo_pelicula_sin_ano.replace(' ', '%20')}&apikey={omdb_api_key}"
    response = requests.get(url)
    data = response.json()
    
    generos = []
    if data.get("Response") == "True":
        generos = data.get("Genre", "").split(", ")
    
    return generos

def obtener_generos_combinados(titulo_pelicula):
    generos = obtener_generos_de_google(titulo_pelicula)
    if not generos:
        generos = obtener_generos_de_omdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de Google y OMDb para '{titulo_pelicula}', probamos con TMDb...")
        generos = obtener_generos_tmdb(titulo_pelicula)
    if not generos:
        print(f"Generos no encontrados a través de TMDb para '{titulo_pelicula}', probamos con IMDb...")
        generos = obtener_generos_imdb(titulo_pelicula)
    if generos:
        print(f"Generos para '{titulo_pelicula}': {', '.join(generos)}")
    else:
        print(f"Generos no encontrados para '{titulo_pelicula}'.")
    return generos

peliculas_lista = [
    "Avatar (2009)", "Avengers: Endgame (2019)", "Avatar: The Way of Water (2022)", "Titanic (1997)", 
    "Star Wars: Episode VII - The Force Awakens (2015)", "Ne Zha 2 (2025) *", "Avengers: Infinity War (2018)", 
    "Spider-Man: No Way Home (2021)", "Inside Out 2 (2024)", "Jurassic World (2015)", "The Lion King (2019)", 
    "The Avengers (2012)", "Furious 7 (2015)", "Top Gun: Maverick (2022)", "Frozen II (2019)", "Barbie (2023)", 
    "Avengers: Age of Ultron (2015)", "Super Mario Bros. Movie (2023)", "Black Panther (2018)", 
    "Harry Potter and the Deathly Hallows: Part 2 (2011)", "Deadpool & Wolverine (2024)", 
    "Star Wars: Episode VIII - The Last Jedi (2017)", "Jurassic World: Fallen Kingdom (2018)", "Frozen (2013)", 
    "Beauty and the Beast (2017)", "Incredibles 2 (2018)", "The Fate of the Furious (2017)", "Iron Man 3 (2013)", 
    "Minions (2015)", "The Lord of the Rings: The Return of the King (2003)", "Captain America: Civil War (2016)", 
    "Aquaman (2018)", "Skyfall (2012)", "Spider-Man: Far From Home (2019)", "Captain Marvel (2019)", 
    "Transformers: Dark of the Moon (2011)", "The Dark Knight Rises (2012)", "Jurassic Park (1993)", 
    "Transformers: Age of Extinction (2014)", "Star Wars: Episode IX - The Rise of Skywalker (2019)", "Joker (2019)", 
    "Toy Story 4 (2019)", "Toy Story 3 (2010)", "Pirates of the Caribbean: Dead Man's Chest (2006)", 
    "Rogue One: A Star Wars Story (2016)", "Moana 2 (2024)", "Aladdin (2019)", "Pirates of the Caribbean: On Stranger Tides (2011)", 
    "Despicable Me 3 (2017)", "Finding Dory (2016)"
]



peliculas_no_encontradas = []
peliculas_con_generos = []

for pelicula in peliculas_lista:
    generos = obtener_generos_combinados(pelicula)
    if not generos:
        peliculas_no_encontradas.append(pelicula)
    else:
        peliculas_con_generos.append([pelicula, ', '.join(generos)])

file_path = r'C:\Users\Usuario\Downloads\progettodef\topboxw.csv'
keys = ['Title', 'Genres']
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(keys)
    writer.writerows(peliculas_con_generos)

print(f"\nArchivo CSV creado exitosamente en: {file_path}")

if peliculas_no_encontradas:
    print("\nLas siguientes películas no fueron encontradas:")
    for movie in peliculas_no_encontradas:
        print(movie)


Generos para 'Avatar (2009)': Action, Science fiction, Fantasy, Adventure, Drama, Classic
Generos para 'Avengers: Endgame (2019)': Action, Superhero, Adventure, Science fiction, Fantasy, Drama
Generos para 'Avatar: The Way of Water (2022)': Animation, Action, Science fiction, Drama, Adventure, Thriller, Fantasy
Generos para 'Titanic (1997)': Romance, Action, Disaster, Melodrama, Adventure, Historical drama, Drama, Thriller, Costume drama
Generos para 'Star Wars: Episode VII - The Force Awakens (2015)': Action, Adventure, Sci-Fi
Generos para 'Ne Zha 2 (2025) *': Animation, Action, Adventure
Generos para 'Avengers: Infinity War (2018)': Action, Superhero, Comedy, Science fiction, Adventure, Drama
Generos para 'Spider-Man: No Way Home (2021)': Action, Romance, Comedy, Superhero, Adventure, Science fiction, Suspense, Drama, Thriller
Generos para 'Inside Out 2 (2024)': Animation, Children's film, Comedy, Adventure, Family film, Coming-of-age story, Drama
Generos para 'Jurassic World (2015)'